In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
# THIS LINE MAKES PLOTS SHOW IN JUPYTER

In [66]:
'''
TO DO: Learn to apply user-defined functions to a data frame

Main ideas:
+ Get comfortable with the GroupBy object -- the  df.groupby(['key1']) before an aggregation function is applied.
+ Apply compiled functions (like mean()) and user-defined functions to a Series or DataFrame
+ Split. Apply. Combine. Split. Apply. Combine.
    Split --> use group by operation, grouped = df.groupby(~~~).  sometimes the grouping operation will require a user-defined function to create the buckets
    Apply --> write a lambda or function (or both) to perform an operation
    Combine --> grouped.apply(~~~)
    
+ Use user-defined functions to establish definitions
    deck is a collection of 52 cards.  Indexed like {Value}{Suit}, so e.g. "2H" or "8C".
    def draw(deck, n=5):
        return draw.take(np.random.permutation(len(deck))[:n])
    get_suit = lambda card: card[-1]
    deck.groupby(get_suit).apply(draw, n=2)
    
# Suppose D is a dict
D.get(x,y) says return D[x] if it exists else return y.  NOT D[y] but just y.
    
***************************

Things I didn't understand but now do:

# What is happening here?  df.apply(regress)
# Explanation: df.apply(regress) uses the df as its paramater, so it's kind of like regress(by_year[index], 'APPL',['SPX']) for each index value
# Group-wise Linear Regression
import statsmodels.api as sm
def regress(data, yvar, xvar):
    Y = data[yvar]
    X = data[xvar]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params
by_year.apply(regress, 'AAPL', ['SPX'])


# How to see if a given Series entry is a substring of one element of a list
employer_set = ['APPLE','MICROSOFT']
fec[fec['contbr_employer_condensed'].str.contains('|'.join(employer_set))][:10]
# Input: '|'.join(employer_set)
# Output: APPLE|MICROSOFT


### I keep getting an error saying that int doesn't have a method 'find'
# It seems like Pandas is trying to operate on the index not the value. The value is a string.

# This does not work
all_contributions['contbr_employer_condensed'].groupby(lambda x: x.find('PALANTIR') > -1).count()

# This does work
all_contributions['contbr_employer_condensed'].apply(lambda x: x.find('PALANTIR') > -1).value_counts()




    
***************************

Nifty:
    df.append([5,6]) will add the list [5,6] as a single value to the end of the data frame
    df.extend([5,6]) will parse the list and add each element to the data frame in order.  So will add element 5 then 6.
    pd.Series.take([1,2,3]) returns the subset of the Series (index + values) at index locations [1,2,3]
        # ONLY WORKS WITH ZERO-INDEXED NATURAL NUMBERS
    df.corrwith()
    Series.corr()


Notes:

-- Each grouping key can take many forms, and the keys do not have to be all of the same type:
    1. A list or array of values that is the same length as the axis being grouped
    2. A value indicating a column name in a DataFrame
    3. A dict or Series giving a correspondence between the values on the axis being grouped and the group names
    4. A function to be invoked on the axis index or the individual labels in the index
    #'s 2-4 are all just ways of producing something that maps to #1

-- df.groupby(['key1'])
    # a GroupBy object that precomputes information about the column key1 in the DataFrame
-- df.groupby(['key1']).sum()
    # apply the function to get computed numbers
    # Result is a series grouped over the unique values of the 'key1' field

-- Grouping supports iteration; works over a sequence of 2-tuples
    for name, group in df.groupby('key1'):
        print(name)
        print(group)
   
-- Difference in:   df.groupby(['key1'])[['data2']].mean()
                    df.groupby(['key1'])['data2'].mean()
                    
                    
-- Grouping with Functions
    - Any function passed as a group key will be called once per index value
    - The return values will be used as the group names

-- Creating a multiIndex & grouping by a level
    columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]], names=['city', 'tenor'])
    hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
    hier_df.groupby(level='city', axis=1).count()

-- user-defined functions applied with "aggregate" (or shorthand as "agg")
    - These are NOT optimized in the way that other functions are

    def peak_to_peak(arr):
        return arr.max() - arr.min()

    grouped = df.groupby('key1')
    grouped.agg(peak_to_peak)

-- Apply multiple functions to a Series or a DataFrame by passing an array
    grouped.agg(['mean','std',peak_to_peak])
    ** DO NOT put user-defined functions in single quotes
    
    - To wind up with a result with a different name than the function, pass a tuple into the agg:
        grouped.agg([('My Mean', 'mean'), ('Peak to Peak', peak_to_peak)])

-- Apply distinct functions to separate columns of a DataFrame
    - Pass a dict into the agg:
        grouped.agg({'col1': 'mean', 'col2' : peak_to_peak'})
    - More than 1 function to a single column
        # The result has hierarchical columns ****
        grouped.agg({'tip_pct' : ['min','max','mean','std',peak_to_peak], 'size': 'sum'})

-- df.transform() broadcasts a scalar value or array of the same size as df back onto df
        df.groupby(key).transform(np.mean) # equivalent to df.groupby(key).transform('mean')
        df.groupby(key).transform(my_func)


-- Combining cut and groupby
    df['data1','data2']
    factor = pd.cut(df['data1'], 4)
        # Cut into 4 equal lenth buckets; factor variable knows the value ranges of the buckets
    grouped = frame['data2'].groupby(factor).describe()
        # Compute statistics about Data 2 using the factor variables
        # Factor is a Category object NOT a list, so we're not just 1-to-1 mapping the results of factor
        # Instead, groupby knows to use the 4 categories of factor to organize the contents of frame['data2']

-- Interpolate missing values
    fill_std = lambda g: g.fillna(g.std())
    data.groupby(group_key).apply(fill_std)

-- Lots of examples in the notes below of the split / apply / combine paradigm

-- Pivot Tables
    - df.pivot_table(index='col1', 'col2')
        #  Default operation is mean.  As in df.pivot_table(index='col1', 'col2', aggfunc = 'mean')
    - tips.pivot_table(['tip_pct','size'], index=['sex', 'day'], columns='smoker', fill_value = 0, aggfunc=sum)
        # margins=True parameter gets the subtotals across each row and column

-- Basic framework

    grouped = df.groupby('name')
    grouped.apply.sum()

--
'''

'\nTO DO: Learn to apply user-defined functions to a data frame\n\nMain ideas:\n+ Get comfortable with the GroupBy object -- the  df.groupby([\'key1\']) before an aggregation function is applied.\n+ Apply compiled functions (like mean()) and user-defined functions to a Series or DataFrame\n+ Split. Apply. Combine. Split. Apply. Combine.\n    Split --> use group by operation, grouped = df.groupby(~~~).  sometimes the grouping operation will require a user-defined function to create the buckets\n    Apply --> write a lambda or function (or both) to perform an operation\n    Combine --> grouped.apply(~~~)\n    \n+ Use user-defined functions to establish definitions\n    deck is a collection of 52 cards.  Indexed like {Value}{Suit}, so e.g. "2H" or "8C".\n    def draw(deck, n=5):\n        return draw.take(np.random.permutation(len(deck))[:n])\n    get_suit = lambda card: card[-1]\n    deck.groupby(get_suit).apply(draw, n=2)\n    \n# Suppose D is a dict\nD.get(x,y) says return D[x] if it ex

In [67]:
# Starting small - create a 5 x 4 data frame

df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one','two','one','two','one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,-0.100674,-1.698715,a,one
1,-1.510790,-0.799394,a,two
2,-0.968428,-1.086428,b,one
3,-1.003918,-0.778593,b,two
4,1.042822,-0.066577,a,one


In [68]:
# Compute mean of the data1 column using the values from the key1 columns
df['data1'].groupby(df['key1'])
    # --> THIS IS A GROUP BY OBJECT.  IT IS NOT YET A NUMBER BECAUSE WE HAVEN'T APPLIED AN OPERATION
    # It has computed intermediate data about the group key df['key1']
    # The object has all the information needed to then apply some operation to each of the groups

In [69]:
# Call an operation on the groupby()
df['data1'].groupby(df['key1']).mean()
    # Produces a new Series that is now indexed by the unique values of the key1 column

key1
a   -0.189547
b   -0.986173
Name: data1, dtype: float64

In [70]:
# Can pass multiple arrays as a list
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means
    # Produces a Series that is HIERARCHICAL index by the unique values of a cartesian product of the key1 & key2 columns

key1  key2
a     one     0.471074
      two    -1.510790
b     one    -0.968428
      two    -1.003918
Name: data1, dtype: float64

In [71]:
# Use "unstack()" to make row index --> column index.
means.unstack()

key2,one,two
key1,,
a,0.471074,-1.510790
b,-0.968428,-1.003918


In [72]:
# Pass two lists to group on as long this works AS LONG AS THEY ARE THE SAME SIZE AS TH EINDEX
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
    # len(states) = 5
years = np.array([2005, 2005, 2006, 2005, 2006])
    # len(years) = 5
df['data1'].groupby([states, years]).mean()

California  2005   -1.510790
            2006   -0.968428
Ohio        2005   -0.552296
            2006    1.042822
Name: data1, dtype: float64

In [73]:
# Pass just a column name if the column to be grouped on is within the initial dataframe / series
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.471074 -0.882646
     two  -1.510790 -0.799394
b    one  -0.968428 -1.086428
     two  -1.003918 -0.778593

In [74]:
# The "size()" metric is like a count on a single column; creates a series, not a DataFrame
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [75]:
##### Grouping supports Iteration
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -0.100674 -1.698715    a  one
1 -1.510790 -0.799394    a  two
4  1.042822 -0.066577    a  one
b
      data1     data2 key1 key2
2 -0.968428 -1.086428    b  one
3 -1.003918 -0.778593    b  two


In [76]:
# You can iterate over multiple keys
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0 -0.100674 -1.698715    a  one
4  1.042822 -0.066577    a  one
('a', 'two')
     data1     data2 key1 key2
1 -1.51079 -0.799394    a  two
('b', 'one')
      data1     data2 key1 key2
2 -0.968428 -1.086428    b  one
('b', 'two')
      data1     data2 key1 key2
3 -1.003918 -0.778593    b  two


In [77]:
# Create a dict from the result of the groupby
pieces = dict(list(df.groupby('key1')))
pieces['b']

,data1,data2,key1,key2
2,-0.968428,-1.086428,b,one
3,-1.003918,-0.778593,b,two


In [78]:
# You can group by dtypes
# -- with axes = 1, basically creates a dict with outer index dtype name and inner index the elements of that dtype
print df.dtypes
grouped = df.groupby(df.dtypes, axis=1)
dict(list(grouped))[df.dtypes[2]]
df

data1    float64
data2    float64
key1      object
key2      object
dtype: object


,data1,data2,key1,key2
0,-0.100674,-1.698715,a,one
1,-1.510790,-0.799394,a,two
2,-0.968428,-1.086428,b,one
3,-1.003918,-0.778593,b,two
4,1.042822,-0.066577,a,one


In [79]:
### Selecting a Column or Subset of Columns and then aggreagating
# Useful for large data sets
df.groupby(['key1', 'key2'])[['data2']].mean()
    # That's processed identically to doing the operation and then selecting the column
    # The interpreter is actually doing the thing above, where only the 1 column to be returned is computed
    # df.groupby(['key1', 'key2']).mean()[['data2']]
    


data2
key1 key2          
a    one  -0.882646
     two  -0.799394
b    one  -1.086428
     two  -0.778593

In [80]:
# Passing a single value selection, e.g. 'data2', instead of a list, ['data2'], yields a Series rather than a DataFrame
df.groupby(['key1', 'key2'])['data2'].mean()

key1  key2
a     one    -0.882646
      two    -0.799394
b     one    -1.086428
      two    -0.778593
Name: data2, dtype: float64

In [81]:
### Grouping with Dicts and Series
people = pd.DataFrame(np.random.randn(5, 5), columns=['a', 'b', 'c', 'd', 'e'], index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.ix[2:3, ['b','c']] = np.nan # Throw in a few NaN values
people

,a,b,c,d,e
Joe,1.333285,-1.619060,0.636468,-1.134022,0.147526
Steve,-1.370139,-0.547247,-1.867878,0.267070,0.256973
Wes,-0.889225,NaN,NaN,0.723350,-0.521414
Jim,0.906904,1.810419,-1.855105,-0.100568,-0.337856
Travis,0.298512,-0.216308,-0.225065,1.210550,-0.593552


In [82]:
# Establish a dict that maps columns of df to other values
mapping = {'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}
    # Not that 'red' and 'blue' are the only values in the codomain when the domain is the columns of df
mapping
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-0.497554,-0.138249
Steve,-1.600807,-1.660413
Wes,0.723350,-1.410639
Jim,-1.955673,2.379467
Travis,0.985485,-0.511347


In [83]:
# Pandas groupby operation gives the same result whether the groupby is passed a dict or a Series
map_series = pd.Series(mapping)
people.groupby(map_series, axis=1).sum()

,blue,red
Joe,-0.497554,-0.138249
Steve,-1.600807,-1.660413
Wes,0.723350,-1.410639
Jim,-1.955673,2.379467
Travis,0.985485,-0.511347


In [84]:
### Grouping with Functions - Can just pass the function name and it will operate once per index value
# For the people names data set above, imagine that we want to group by the length of the name.
people.groupby(len).sum()

,a,b,c,d,e
3,1.350964,0.191359,-1.218638,-0.511239,-0.711743
5,-1.370139,-0.547247,-1.867878,0.267070,0.256973
6,0.298512,-0.216308,-0.225065,1.210550,-0.593552


In [85]:
key_list = ['one', 'one', 'one', 'two', 'two']
    # len of index are (3, 5, 3, 3, 5) so there are 3/one and 3/two
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.889225 -1.619060  0.636468 -1.134022 -0.521414
  two  0.906904  1.810419 -1.855105 -0.100568 -0.337856
5 one -1.370139 -0.547247 -1.867878  0.267070  0.256973
6 two  0.298512 -0.216308 -0.225065  1.210550 -0.593552

In [86]:
# Grouping by Index Levels
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], [1, 3, 5, 1, 3]], names=['city', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      0.407177  1.201516  0.418219  0.738341  0.372628
1     -0.528032 -2.874074  0.326185 -0.956307 -0.613605
2      0.138301  0.329221 -1.058965  0.720067 -0.209493
3      1.156720 -1.085918  1.006451 -0.158568  0.060004

In [87]:
hier_df.groupby(level=['tenor'], axis=1).count()

tenor,1,3,5
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1


In [88]:
### Data Aggregation
df.groupby('key1')['data1'].quantile(0.9)

key1
a    0.814123
b   -0.971977
Name: data1, dtype: float64

In [89]:
# User-defined function -- pass into the 'aggregate' or 'agg' method
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped = df.groupby('key1')
grouped.aggregate(peak_to_peak)

,data1,data2
key1,,
a,2.553611,1.632138
b,0.035490,0.307835


In [90]:
# Describe works
grouped.describe()

data1     data2
key1                          
a    count  3.000000  3.000000
     mean  -0.189547 -0.854896
     std    1.279123  0.817483
     min   -1.510790 -1.698715
     25%   -0.805732 -1.249055
     50%   -0.100674 -0.799394
     75%    0.471074 -0.432986
     max    1.042822 -0.066577
b    count  2.000000  2.000000
     mean  -0.986173 -0.932510
     std    0.025095  0.217672
     min   -1.003918 -1.086428
     25%   -0.995046 -1.009469
     50%   -0.986173 -0.932510
     75%   -0.977301 -0.855552
     max   -0.968428 -0.778593

In [91]:
#### Multiple column aggregation
# With Series, you can apply different functions to the value data which, recall, must have just 1 value column
# With DataFrames, you can apply different functions to different columns

In [92]:

tips = pd.read_csv('ch08/tips.csv')

# Tips as pctg of total bill
# Adding a new column to a dataframe
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [93]:
# Make preliminary groupings for use with functions later
grouped = tips.groupby(['sex','smoker'])
grouped_pct = grouped['tip_pct']

In [94]:
# the aggregate() function can take compiled function in addition to user-defined functions
# IMPORTANT: user-defined functions have no quotes.  

def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
sex    smoker                                  
Female No      0.156921  0.036421      0.195876
       Yes     0.182150  0.071595      0.360233
Male   No      0.160669  0.041849      0.220186
       Yes     0.152771  0.090588      0.674707

In [95]:
# You can give columns based on functions other names -- especially helpful for passing in Lambda functions
# Pass a tuple into the agg
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
sex    smoker                    
Female No      0.156921  0.036421
       Yes     0.182150  0.071595
Male   No      0.160669  0.041849
       Yes     0.152771  0.090588

In [96]:
## DataFrames that apply multiple functions to multiple columns automatically create a hierarchical index
# The resulting DataFrame can be accessed like any other data frame
functions = ['count', 'mean', 'max']
result = grouped[['tip_pct', 'total_bill']].agg(functions)
result

tip_pct                     total_bill                  
                count      mean       max      count       mean    max
sex    smoker                                                         
Female No          54  0.156921  0.252672         54  18.105185  35.83
       Yes         33  0.182150  0.416667         33  17.977879  44.30
Male   No          97  0.160669  0.291990         97  19.791237  48.33
       Yes         60  0.152771  0.710345         60  22.284500  50.81

In [97]:
# Multiple functions to the same columns, then assigning the resulting functions new names
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped[['tip_pct', 'total_bill']].agg(ftuples)

tip_pct              total_bill           
              Durchschnitt Abweichung Durchschnitt Abweichung
sex    smoker                                                
Female No         0.156921   0.001327    18.105185  53.092422
       Yes        0.182150   0.005126    17.977879  84.451517
Male   No         0.160669   0.001751    19.791237  76.152961
       Yes        0.152771   0.008206    22.284500  98.244673

In [98]:
# Applying functions to *separate* columns
# Pass a dict into the agg
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
sex    smoker            
Female No       5.2   140
       Yes      6.5    74
Male   No       9.0   263
       Yes     10.0   150

In [99]:
# You can apply more than 1 function at a time to each column
grouped.agg({'tip_pct' : ['min','max','mean','std',peak_to_peak], 'size': 'sum'})

tip_pct                                            size
                    min       max      mean       std peak_to_peak  sum
sex    smoker                                                          
Female No      0.056797  0.252672  0.156921  0.036421     0.195876  140
       Yes     0.056433  0.416667  0.182150  0.071595     0.360233   74
Male   No      0.071804  0.291990  0.160669  0.041849     0.220186  263
       Yes     0.035638  0.710345  0.152771  0.090588     0.674707  150

In [100]:
### Group-wide Operations and Transformations
# More general operations than reduction operations where a 1-D vector becomes a scalar

In [101]:
## E.g. add the group mean over an index as its own column onto a dataframe
# Iterative method with a groupby over the df and then a merge back into the original DF:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
    # Returns a DataFrame with an index of key1's unique value: [a, b]
pd.merge(df, k1_means, left_on='key1', right_index=True)
    # Says to merge using the key1 column of df (the "left" df) and the index of k1_means (the "right" df)

,data1,data2,key1,key2,mean_data1,mean_data2
0,-0.100674,-1.698715,a,one,-0.189547,-0.854896
1,-1.510790,-0.799394,a,two,-0.189547,-0.854896
4,1.042822,-0.066577,a,one,-0.189547,-0.854896
2,-0.968428,-1.086428,b,one,-0.986173,-0.932510
3,-1.003918,-0.778593,b,two,-0.986173,-0.932510


In [102]:
# Use a the "transform" method to apply a function a.k.a. "broadcast" to elements of a group

# See the values you'll be broadcasting
key = ['one', 'two', 'one', 'two', 'one']
people.groupby(key).mean()

,a,b,c,d,e
one,0.247524,-0.917684,0.205701,0.266626,-0.322480
two,-0.231617,0.631586,-1.861492,0.083251,-0.040442


In [103]:
# Broadcast them

# With "Transform" the operation over the group is calculated and 
# the result is put back into the original array depending on which group the cell was in
# ()"broadcasted" onto the DataFrame)
people.groupby(key).transform(np.mean)
    # Notice that the values above show up below in the specified order

,a,b,c,d,e
Joe,0.247524,-0.917684,0.205701,0.266626,-0.322480
Steve,-0.231617,0.631586,-1.861492,0.083251,-0.040442
Wes,0.247524,-0.917684,0.205701,0.266626,-0.322480
Jim,-0.231617,0.631586,-1.861492,0.083251,-0.040442
Travis,0.247524,-0.917684,0.205701,0.266626,-0.322480


In [104]:
# Broadcast with a user-defined function
def demean(arr):
    return arr - arr.mean()

demeaned = people.groupby(key).transform(demean)
print demeaned

# Can verify that the average value of the resulting transformed df is now zero, as expected
print demeaned.groupby(key).mean()

               a         b         c         d         e
Joe     1.085761 -0.701376  0.430766 -1.400648  0.470006
Steve  -1.138521 -1.178833 -0.006386  0.183819  0.297415
Wes    -1.136749       NaN       NaN  0.456724 -0.198934
Jim     1.138521  1.178833  0.006386 -0.183819 -0.297415
Travis  0.050988  0.701376 -0.430766  0.943924 -0.271072
                a             b    c             d    e
one -1.850372e-17 -5.551115e-17  0.0  3.700743e-17  0.0
two  0.000000e+00 -1.110223e-16  0.0  0.000000e+00  0.0


In [105]:
# Apply is the most general groupby method
    # Splits the object being manipulated into pieces
    # Invokes the function over each pieces
    # Concatenates the pieces together
    # "Split, Apply, Combined" is the framework
    
# Select the top 5 tip_pct values by group

# The general function we want to apply:
def top(df, n=5, column = 'tip_pct'):
    return df.sort_values(by=column)[-n:]

In [106]:
# This selects the top 5 across the total data set
top(tips, n=6, column='tip_pct')

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [107]:
# This selects the top 5 across smokers 
# -- the function above is applied to each group object
# -- and then glued together using pd.concat()
tips.groupby(['smoker']).apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

In [108]:
# Can do multi-level group by, as well as pass the other parameters of the called function in with the apply
tips.groupby(['smoker','day']).apply(top, n=1, column='total_bill')
    # Can suppress the new index by setting group_keys=False in the groupby(), but don't do that. 

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

In [109]:
### Combining GroupBy and Cut functions
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
    # Two columns: data1 and data2 with 1000 items
factor = pd.cut(frame.data1, 4)
    # Bucket into -- maps the object in factor.data1 into 1 of 4 identically sized buckets
    # The resulting object is an array of the buckets that each original value fell into
factor

0      (-1.607, 0.0439]
1       (0.0439, 1.694]
2      (-3.264, -1.607]
3      (-1.607, 0.0439]
4       (0.0439, 1.694]
5      (-1.607, 0.0439]
6       (0.0439, 1.694]
7      (-1.607, 0.0439]
8      (-1.607, 0.0439]
9       (0.0439, 1.694]
10      (0.0439, 1.694]
11     (-1.607, 0.0439]
12     (-1.607, 0.0439]
13     (-1.607, 0.0439]
14      (0.0439, 1.694]
15     (-1.607, 0.0439]
16      (0.0439, 1.694]
17     (-1.607, 0.0439]
18      (0.0439, 1.694]
19       (1.694, 3.345]
20      (0.0439, 1.694]
21      (0.0439, 1.694]
22      (0.0439, 1.694]
23      (0.0439, 1.694]
24      (0.0439, 1.694]
25     (-1.607, 0.0439]
26     (-3.264, -1.607]
27     (-3.264, -1.607]
28     (-1.607, 0.0439]
29     (-1.607, 0.0439]
             ...       
970      (1.694, 3.345]
971     (0.0439, 1.694]
972     (0.0439, 1.694]
973    (-1.607, 0.0439]
974      (1.694, 3.345]
975     (0.0439, 1.694]
976    (-1.607, 0.0439]
977     (0.0439, 1.694]
978     (0.0439, 1.694]
979     (0.0439, 1.694]
980     (0.0439,

In [110]:
# We can use the factor array as the grouping set
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

# Compute statistics about Data 2 using the factor variables
grouped = frame['data2'].groupby(factor)
    # Factor is a Category object NOT a list, so we're not just 1-to-1 mapping the results of factor
    # Instead, groupby knows to use the 4 categories of factor to organize the contents of frame['data2']

grouped.apply(get_stats) # "Give me summary statistics over the data frame bucketing by the ranges in group 1"

data1                  
(-3.264, -1.607]  count     56.000000
                  max        2.444516
                  mean      -0.007081
                  min       -2.356510
(-1.607, 0.0439]  count    432.000000
                  max        2.572137
                  mean       0.000661
                  min       -3.271478
(0.0439, 1.694]   count    471.000000
                  max        2.834259
                  mean       0.060786
                  min       -3.082020
(1.694, 3.345]    count     41.000000
                  max        2.307908
                  mean       0.206024
                  min       -1.940212
Name: data2, dtype: float64

In [111]:
grouping = pd.qcut(frame['data1'], 10, labels=False)
    # labels = False --> just broadcasts the ordinal number of the bucket for the value, not a tuple of where the value's range falls.
grouped = frame['data2'].groupby(grouping)
grouped.describe().unstack()
    # Or use the method we made, get_stats, that's a subset of quip's functionality.

,count,mean,std,min,25%,50%,75%,max
data1,,,,,,,,
0,100.0,-0.050650,1.001775,-2.356510,-0.593061,-0.139919,0.505763,2.444516
1,100.0,0.033671,0.991148,-2.505588,-0.533259,-0.007762,0.712738,2.357573
2,100.0,-0.049138,0.976272,-2.532918,-0.702647,-0.042394,0.400940,2.538713
3,100.0,0.137220,0.958119,-1.993263,-0.618779,0.243603,0.840936,2.572137
4,100.0,-0.066161,1.133679,-3.271478,-0.822328,-0.104646,0.804354,2.185852
5,100.0,0.109980,1.146366,-2.491876,-0.736789,0.130848,0.833796,2.834259
6,100.0,-0.056062,0.942486,-2.055331,-0.705593,0.009946,0.610140,2.254661
7,100.0,0.177467,0.943340,-1.698190,-0.455874,0.053179,0.917012,2.128700
8,100.0,0.069969,1.143650,-3.082020,-0.541785,0.064257,0.860941,2.694687


In [112]:
## Example: Filling missing values with Group-specific values

# Set every other value to NaN
s = pd.Series(np.random.randn(6))
s[::2] = np.nan # Extended slices.  Neat.
s

0         NaN
1    0.475471
2         NaN
3    0.783362
4         NaN
5   -0.072427
dtype: float64

In [113]:
# Fill in those NaN's with the population mean
s.fillna(s.mean())

0    0.395469
1    0.475471
2    0.395469
3    0.783362
4    0.395469
5   -0.072427
dtype: float64

In [114]:
# Fill in the NaN's with means based on grouping of the data
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.723663
New York      0.596251
Vermont            NaN
Florida       0.463986
Oregon        1.149414
Nevada             NaN
California   -0.268151
Idaho              NaN
dtype: float64

In [115]:
data.groupby(group_key).mean()

East    0.594633
West    0.440632
dtype: float64

In [116]:
# Use a lambda because it's such a simple function

fill_mean = lambda g: g.fillna(g.mean())

# data.groupby(group_key) --> "create my buckets defined by the group keys"
# .apply(fill_mean) --> send each bucket group to the fill_mean function
# As written above, the fill_mean function will fill the na values in that passed group with the mean of that passed group
data.groupby(group_key).apply(fill_mean)

Ohio          0.723663
New York      0.596251
Vermont       0.594633
Florida       0.463986
Oregon        1.149414
Nevada        0.440632
California   -0.268151
Idaho         0.440632
dtype: float64

In [117]:
# Exercise 1b.  Fill in each NA with the standard deviation:

# The below one-liner is equivalent to what's shown below; it's also reusable, so do it that way
#data.groupby(group_key).apply(lambda g: g.fillna(g.std()))

# The above one-liner is equivalent to what's shown below
fill_std = lambda g: g.fillna(g.std())
data.groupby(group_key).apply(fill_std)

Ohio          0.723663
New York      0.596251
Vermont       0.129846
Florida       0.463986
Oregon        1.149414
Nevada        1.002370
California   -0.268151
Idaho         1.002370
dtype: float64

In [118]:
### Random sampling and permutation

# Playing cards
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
# Create the numerical range of card values
card_val = (range(1, 11) + [10] * 3) * 4
# Create the human quasi-numerical of the cards
base_names = ['A'] + range(2, 11) + ['J', 'K', 'Q']
# Create the cards themselves by parsing the human value + suit
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
# Create the deck by making the suit
deck = pd.Series(card_val, index=cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [119]:
# Draw a specific number of cards

def draw(deck, n=5):
    # np.random.permutation(x) Get a non-repeating list of natural numbers 0 through x-1
    return deck.take(np.random.permutation(len(deck)))[:n]
    
draw(deck, 5)

7S      7
AD      1
10D    10
8C      8
4H      4
dtype: int64

In [120]:
# Draw a specific number of cards from each suit

# Find the suit of the card, which is the last letter of the suit
get_suit = lambda card: card[-1]

# Group over the suit value. We need a user-defined function to define the group population
# Here we're saying "establish a population grouping levels by seeing the last item in each string"
# Then to each grouping apply the draw function
deck.groupby(get_suit).apply(draw,n=2)

C  9C     9
   4C     4
D  7D     7
   KD    10
H  3H     3
   KH    10
S  7S     7
   8S     8
dtype: int64

In [121]:
# To summarize - group by a lambda and apply a user-defined function
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck)))[:n]
get_suit = lambda card: card[-1]
deck.groupby(get_suit).apply(draw, n=2)

C  9C      9
   5C      5
D  10D    10
   2D      2
H  8H      8
   JH     10
S  2S      2
   5S      5
dtype: int64

In [122]:
# Group weighted average and correlation
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b','b'],
                  'data': np.random.randn(8),
                  'weights': np.random.rand(8)})

df

,category,data,weights
0,a,1.927733,0.161918
1,a,1.073884,0.902264
2,a,-0.374243,0.098370
3,a,0.004956,0.345388
4,b,-1.113687,0.950988
5,b,-0.173991,0.961219
6,b,-3.532274,0.252321
7,b,-1.763844,0.072591


In [123]:
# Group by weight - average 'data' field by by category value
# Split 
grouped = df.groupby('category')

# Apply - 'weights' is a parameter of the np.average() method.
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])

# Combine
grouped.apply(get_wavg)


category
a    0.826266
b   -1.003815
dtype: float64

In [124]:
# End of day stock prices on the S&P 500
close_px = pd.read_csv('ch09/stock_px.csv', parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [125]:
# See first 4 values
close_px[:4]

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93


In [126]:
# Find correlation of the stocks with the SP500 index of stocks

## Methods used here:
#  pct_change() - broadcasts the percent change from the previous cell to the next cell, using index of next cell.
#  corrwith()


rets = close_px.pct_change().dropna()
spx_corr = lambda x: x.corrwith(x['SPX'])
by_year = rets.groupby(lambda x: x.year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [127]:
# Similarly, using the same dataset find the correlations of Apple with Microsoft
aapl_msft_corr = lambda x: x.corrwith(x['MSFT'])
by_year.apply(aapl_msft_corr)['AAPL']
#by_year = rets.groupby(lambda x: x.year)
#by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
Name: AAPL, dtype: float64

In [130]:
# Group-wise Linear Regression
import statsmodels.api as sm
def regress(data, yvar, xvar):
    Y = data[yvar]
    X = data[xvar]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


In [131]:
# Pivot Tables and Cross-Tabulation
# Use the tips DataFrame with the derived tip_pct column above

print tips[:5]
tips.pivot_table(index=['sex', 'smoker'], )
    # The default aggregaton is means.

   total_bill   tip     sex smoker  day    time  size   tip_pct
0       16.99  1.01  Female     No  Sun  Dinner     2  0.059447
1       10.34  1.66    Male     No  Sun  Dinner     3  0.160542
2       21.01  3.50    Male     No  Sun  Dinner     3  0.166587
3       23.68  3.31    Male     No  Sun  Dinner     2  0.139780
4       24.59  3.61  Female     No  Sun  Dinner     4  0.146808


size       tip   tip_pct  total_bill
sex    smoker                                          
Female No      2.592593  2.773519  0.156921   18.105185
       Yes     2.242424  2.931515  0.182150   17.977879
Male   No      2.711340  3.113402  0.160669   19.791237
       Yes     2.500000  3.051167  0.152771   22.284500

In [132]:
# Pivot Table specifications:
#   Aggregate only the "tip_pct" and "size" columns
#  Want to group over "day", "sex", and "smoker" 

tips.pivot_table(['tip_pct','size'], index=['sex', 'smoker', 'day'])

size   tip_pct
sex    smoker day                     
Female No     Fri   2.500000  0.165296
              Sat   2.307692  0.147993
              Sun   3.071429  0.165710
              Thur  2.480000  0.155971
       Yes    Fri   2.000000  0.209129
              Sat   2.200000  0.163817
              Sun   2.500000  0.237075
              Thur  2.428571  0.163073
Male   No     Fri   2.000000  0.138005
              Sat   2.656250  0.162132
              Sun   2.883721  0.158291
              Thur  2.500000  0.165706
       Yes    Fri   2.125000  0.144730
              Sat   2.629630  0.139067
              Sun   2.600000  0.173964
              Thur  2.300000  0.164417

In [133]:
# For ease of parsing we can put the "smoker" data in the columns
# Add margins=True parameter to get the subtotals across each row and column
tips.pivot_table(['tip_pct','size'], index=['sex', 'day'], columns='smoker', margins = True)

tip_pct                          size                    
smoker             No       Yes       All        No       Yes       All
sex    day                                                             
Female Fri   0.165296  0.209129  0.199388  2.500000  2.000000  2.111111
       Sat   0.147993  0.163817  0.156470  2.307692  2.200000  2.250000
       Sun   0.165710  0.237075  0.181569  3.071429  2.500000  2.944444
       Thur  0.155971  0.163073  0.157525  2.480000  2.428571  2.468750
Male   Fri   0.138005  0.144730  0.143385  2.000000  2.125000  2.100000
       Sat   0.162132  0.139067  0.151577  2.656250  2.629630  2.644068
       Sun   0.158291  0.173964  0.162344  2.883721  2.600000  2.810345
       Thur  0.165706  0.164417  0.165276  2.500000  2.300000  2.433333
All          0.159328  0.163196  0.160803  2.668874  2.408602  2.569672

In [134]:
# pass a different aggregation function
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'smoker'], columns='day', aggfunc=len, margins=True)

tip_pct                           size                         
day               Fri   Sat   Sun  Thur    All   Fri   Sat   Sun  Thur    All
sex    smoker                                                                
Female No         2.0  13.0  14.0  25.0   54.0   2.0  13.0  14.0  25.0   54.0
       Yes        7.0  15.0   4.0   7.0   33.0   7.0  15.0   4.0   7.0   33.0
Male   No         2.0  32.0  43.0  20.0   97.0   2.0  32.0  43.0  20.0   97.0
       Yes        8.0  27.0  15.0  10.0   60.0   8.0  27.0  15.0  10.0   60.0
All              19.0  87.0  76.0  62.0  244.0  19.0  87.0  76.0  62.0  244.0

In [135]:
# fill_value parameter replaces NaN
tips.pivot_table(['size'], index=['time', 'sex', 'smoker'], columns='day', aggfunc='sum', fill_value=0)

size              
day                   Fri Sat  Sun Thur
time   sex    smoker                   
Dinner Female No        2  30   43    2
              Yes       8  33   10    0
       Male   No        4  85  124    0
              Yes      12  71   39    0
Lunch  Female No        3   0    0   60
              Yes       6   0    0   17
       Male   No        0   0    0   50
              Yes       5   0    0   23

In [136]:
# Cross-Tabulations: Crosstab -- Computes Group Frequencies
data = pd.DataFrame({'Sample': range(1,11),
                     'Gender': ['Female', 'Male', 'Female', 'Male', 'Male', 'Male', 'Female', 'Female', 'Male', 'Female'],
                     'Handedness': ['Right-handed', 'Left-handed', 'Right-handed', 'Right-handed', 'Left-handed', 'Right-handed', 'Right-handed', 'Left-handed', 'Right-handed', 'Right-handed']})
data

,Gender,Handedness,Sample
0,Female,Right-handed,1
1,Male,Left-handed,2
2,Female,Right-handed,3
3,Male,Right-handed,4
4,Male,Left-handed,5
5,Male,Right-handed,6
6,Female,Right-handed,7
7,Female,Left-handed,8
8,Male,Right-handed,9
9,Female,Right-handed,10


In [137]:
# Crosstab simplifies pivot table opeations
pd.crosstab(data.Gender, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Gender,,,
Female,1,4,5
Male,2,3,5
All,3,7,10


In [138]:
# But don't be a lazy bastard, just use a Pivot Table
data.pivot_table(index=['Gender'], columns=['Handedness'], margins=True)['Sample']

Handedness,Left-handed,Right-handed,All
Gender,,,
Female,8.0,5.250000,5.8
Male,3.5,6.333333,5.2
All,5.0,5.714286,5.5


In [140]:
# 2016 Federal Election Commission
# National data is 1.46 GB, loaded into memory is > 3 GB

fec = pd.read_csv('~/Downloads/P00000001-ALL.csv', index_col = None)
# The first column is being misinterpreted as the index because it doesn't have quotes around it
fec = fec.reset_index()
columns_list = fec.columns
columns_list = columns_list[1:]
# columns_list.values
del fec['election_tp']
fec.columns = columns_list
fec[:5]

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cmte_id,cand_id,cand_nm,contbr_nm,contbr_city,contbr_st,contbr_zip,contbr_employer,contbr_occupation,contb_receipt_amt,contb_receipt_dt,receipt_desc,memo_cd,memo_text,form_tp,file_num,tran_id,election_tp
0,C00458844,P60006723,"Rubio, Marco","BLUM, MAUREEN",WASHINGTON,20,DC,STRATEGIC COALITIONS & INITIATIVES LL,OUTREACH DIRECTOR,175.0,15-MAR-16,NaN,NaN,NaN,SA17A,1082559,SA17.1152124,P2016
1,C00458844,P60006723,"Rubio, Marco","DODSON, MARK B. MR.",ATLANTA,30,GA,MORTGAGE CAPITAL ADVISORS,PRIVATE MORTGAGE BANKING,25.0,16-MAR-16,NaN,NaN,DEBT RETIREMENT,SA17A,1082559,SA17.1153048,P2016
2,C00458844,P60006723,"Rubio, Marco","CHILDERS, WILLIAM",DPO,AE,098309998,DIPLOMAT,US GOVERNMENT,100.0,20-FEB-16,NaN,NaN,NaN,SA17A,1056862,SA17.1020839,P2016
3,C00458844,P60006723,"Rubio, Marco","RUCINSKI, ROBERT",APO,AE,090960009,US ARMY,PHYSICIAN,200.0,10-MAR-16,NaN,NaN,NaN,SA17A,1082559,SA17.1078677,P2016
4,C00458844,P60006723,"Rubio, Marco","RUCINSKI, ROBERT",APO,AE,090960009,US ARMY,PHYSICIAN,100.0,08-MAR-16,NaN,NaN,NaN,SA17A,1082559,SA17.1074981,P2016


In [141]:
# Get unique list of political candidates
unique_cands = fec.cand_nm.unique()
unique_cands

array(['Rubio, Marco', 'Santorum, Richard J.', 'Perry, James R. (Rick)',
       'Carson, Benjamin S.', "Cruz, Rafael Edward 'Ted'", 'Paul, Rand',
       'Clinton, Hillary Rodham', 'Sanders, Bernard', 'Fiorina, Carly',
       'Huckabee, Mike', 'Pataki, George E.', "O'Malley, Martin Joseph",
       'Graham, Lindsey O.', 'Bush, Jeb', 'Trump, Donald J.',
       'Jindal, Bobby', 'Christie, Christopher J.', 'Walker, Scott',
       'Stein, Jill', 'Webb, James Henry Jr.', 'Kasich, John R.',
       'Gilmore, James S III', 'Lessig, Lawrence', 'Johnson, Gary',
       'McMullin, Evan'], dtype=object)

In [142]:
# Manually append the Parties information to this
parties = {'Rubio, Marco': 'Republican',
 'Santorum, Richard J.': 'Republican',
 'Perry, James R. (Rick)': 'Republican',
 'Carson, Benjamin S.': 'Republican',
 "Cruz, Rafael Edward 'Ted'": 'Republican',
 'Paul, Rand': 'Republican',
'Clinton, Hillary Rodham': 'Democratic',
 'Sanders, Bernard': 'Democratic',
 'Fiorina, Carly': 'Republican',
 'Huckabee, Mike': 'Republican',
 'Pataki, George E.': 'Republican',
 "O'Malley, Martin Joseph": 'Democratic',
 'Graham, Lindsey O.': 'Republican',
 'Bush, Jeb': 'Republican',
 'Trump, Donald J.': 'Republican',
 'Jindal, Bobby': 'Republican',
 'Christie, Christopher J.': 'Republican',
 'Walker, Scott': 'Republican',
 'Stein, Jill': 'Green',
 'Webb, James Henry Jr.': 'Democratic',
 'Kasich, John R.': 'Republican',
 'Gilmore, James S III': 'Republican',
 'Lessig, Lawrence': 'Independent',
 'Johnson, Gary': 'Libertarian',
 'McMullin, Evan': 'Independent'}
parties

{'Bush, Jeb': 'Republican',
 'Carson, Benjamin S.': 'Republican',
 'Christie, Christopher J.': 'Republican',
 'Clinton, Hillary Rodham': 'Democratic',
 "Cruz, Rafael Edward 'Ted'": 'Republican',
 'Fiorina, Carly': 'Republican',
 'Gilmore, James S III': 'Republican',
 'Graham, Lindsey O.': 'Republican',
 'Huckabee, Mike': 'Republican',
 'Jindal, Bobby': 'Republican',
 'Johnson, Gary': 'Libertarian',
 'Kasich, John R.': 'Republican',
 'Lessig, Lawrence': 'Independent',
 'McMullin, Evan': 'Independent',
 "O'Malley, Martin Joseph": 'Democratic',
 'Pataki, George E.': 'Republican',
 'Paul, Rand': 'Republican',
 'Perry, James R. (Rick)': 'Republican',
 'Rubio, Marco': 'Republican',
 'Sanders, Bernard': 'Democratic',
 'Santorum, Richard J.': 'Republican',
 'Stein, Jill': 'Green',
 'Trump, Donald J.': 'Republican',
 'Walker, Scott': 'Republican',
 'Webb, James Henry Jr.': 'Democratic'}

In [143]:
# Map these values to the party
# Example:
fec['cand_nm'][11111:11116].map(parties)

11111    Republican
11112    Republican
11113    Republican
11114    Republican
11115    Republican
Name: cand_nm, dtype: object

In [144]:
# Create the party as a new column
fec['party'] = fec['cand_nm'].map(parties)

In [ ]:
# Only count contributions > 0
(fec.contb_receipt_amt > 0).value_counts()
fec = fec[fec.contb_receipt_amt > 0]

In [ ]:
# Only count donations to Hillary, Sanders, Trump, and Cruz
fec_leaders = fec[fec['cand_nm'].isin(['Trump, Donald J.', 'Clinton, Hillary Rodham', 'Sanders, Bernard', "Cruz, Rafael Edward 'Ted'"])]
fec_leaders['cand_nm'].unique()

In [ ]:
# Donations by occupation to the leaders
fec_leaders['contbr_occupation'].value_counts()[:100]

In [ ]:
# Create dicts to enable mapping the same values
occ_mapping = {
    'INFORMATION REQUESTED PER BEST EFFORTS': 'NOT PROVIDED',
    'INFORMATION REQUESTED': 'NOT PROVIDED',
    'INFORMATION REQUESTED (BEST EFFORTS)': 'NOT PROVIDED',
    'C.E.O.': 'CEO',
    'TEACHER': 'EDUCATOR',
    'SELF EMPLOYED':'SELF-EMPLOYED' 
}

emp_mapping = {
    'INFORMATION REQUESTED PER BEST EFFORTS': 'NOT PROVIDED',
    'INFORMATION REQUESTED': 'NOT PROVIDED',
    'INFORMATION REQUESTED (BEST EFFORTS)': 'NOT PROVIDED',
    'SELF EMPLOYED':'SELF-EMPLOYED',
    'SELF': 'SELF-EMPLOYED',
    'SELF EMPLOYED-EMPLOYED': 'SELF-EMPLOYED',
    'NONE': 'NOT PROVIDED',
    'UNEMPLOYED': 'NOT EMPLOYED',
    'U.S. POSTAL SERVICE': 'USPS',
    'MR.': 'NOT PROVIDED',
    'MS.': 'NOT PROVIDED',
    'PALANTIR TECHNOLOGIES':'PALANTIR',
    'PALANTIR TECHNOLOGIES, INC.':'PALANTIR',
    'PALANTIR TECHNOLOGIES, INC':'PALANTIR',
    'PALANTIR USG':'PALANTIR',
}


f = lambda x: occ_mapping.get(x, x)
g = lambda x: emp_mapping.get(x, x)
# D.get(x,y) says return D[x] if it exists else return y.  NOT D[y] but just y.

fec['contbr_occuption_condensed'] =fec['contbr_occupation'].map(f)
fec['contbr_employer_condensed'] = fec['contbr_employer'].map(g)
fec_leaders['contbr_occuption_condensed'] = fec_leaders['contbr_occupation'].map(f)
fec_leaders['contbr_employer_condensed'] = fec_leaders['contbr_employer'].map(g)

#fec['contbr_occuption_condensed'].value_counts()[:10]
# fec['contbr_employer_condensed'].value_counts()[:200]

In [ ]:
# Find all occupations donating > $1 M
by_occupation = fec.pivot_table('contb_receipt_amt', index='contbr_occuption_condensed',columns='party', aggfunc='sum')
by_occupation[by_occupation.sum(1) > 5000000][['Democratic','Republican']].plot(kind='barh')

In [ ]:
# Find top occupations and employers donating to each candidate
def get_top_amounts(group, key, n=5):
    # Order totals by key in descending order
    return group.groupby(key)['contb_receipt_amt'].sum().sort_values(ascending=False)[:n]

# Split -- split by the cand_nm field
grouped = fec_leaders.groupby('cand_nm')

In [ ]:
# See top 7 contributors by occupation
# Apply & Combine
grouped.apply(get_top_amounts, 'contbr_occuption_condensed', n=7)

In [ ]:
# See top 7 contributors by employer
# Apply & Combine
grouped.apply(get_top_amounts, 'contbr_employer_condensed', n=20)

In [ ]:
# Analyze by logarithmic buckets
bins = np.array([0, 1, 10, 26, 27, 100, 1000, 10000, 100000, 1000000, 10000000])
labels = pd.cut(fec_leaders['contb_receipt_amt'], bins)

# Group by the buckets & each candidate name
grouped = fec_leaders.groupby(['cand_nm', labels])
grouped.size().unstack(0)

In [ ]:
bucket_sums = grouped['contb_receipt_amt'].sum().unstack(0)
bucket_sums

In [ ]:
total_in_ranges = bucket_sums.sum(axis=1) # sum over all candidates by dollar range (sum over the columns bucketing by the rows)
total_in_ranges
bucket_sums.div(total_in_ranges, axis=0) # the axis passed tells which axis matches.  

In [ ]:
# Aggregate by state

# Split
grouped = fec_leaders.groupby(['cand_nm','contbr_st'])

# Combined
grouped.size().unstack(0) # 0 is the level passed

per_state = grouped['contb_receipt_amt'].sum().unstack(0).fillna(0)
total_by_state = per_state.sum(axis=1)
per_state.div(total_by_state, axis=0)